In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from keras.preprocessing import image

In [2]:
model = load_model(r'C:\Users\admin\Downloads\Masks\CNN\mask_predict.h5')

harr_path = r'C:\Users\admin\Downloads\Masks\Cascades\lbpcascades\lbpcascade_frontalface.xml'
cascade = cv2.CascadeClassifier(harr_path)

In [3]:
def detect_face_mask_video(img):
    
    global model
        
    shift = 60
    
    img_gray = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
    dims = cascade.detectMultiScale(img_gray)
    
    for (x,y,w,h) in dims:
        
        roi = img[y-shift:y+h+shift , x-shift:x+w+shift]
        roi = cv2.cvtColor(roi , cv2.COLOR_BGR2RGB)
        roi = cv2.resize(roi , dsize=(256,256))
        Class = model.predict(tf.expand_dims(roi , axis=0))
                
        if (Class[0] == 1):
            cv2.rectangle(img , (x,y) , (x+w,y+h) , (0,0,255) , 2)     # No Masks
            cv2.putText(img, text='UNSAFE', org=(x,y-2), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,0,255), thickness=4, lineType=cv2.LINE_AA)
        else:
            cv2.rectangle(img , (x,y) , (x+w,y+h) , (0,255,0) , 2)     # Masks
            cv2.putText(img, text='SAFE', org=(x,y-2), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0), thickness=4, lineType=cv2.LINE_AA)
    
    return img

In [4]:
cam = cv2.VideoCapture(0)

width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

writer = cv2.VideoWriter(r'Desktop\My Video.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 20, (width, height))

while True:
    
    _, frame = cam.read()
    frame = detect_face_mask_video(cv2.flip(frame , 1))
    writer.write(frame)
    cv2.imshow("Web Cam" , frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
writer.release()
cv2.destroyAllWindows()